In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

In [3]:
import matplotlib.pylab as plt
import numpy as np
import pickle, os, warnings, sys, random, logging, librosa, json, nemo
import soundfile as sf
from tqdm.auto import tqdm
from ruamel.yaml import YAML
import nemo.collections.asr as nemo_asr
from omegaconf import DictConfig
import pytorch_lightning as pl
from Cfg import Cfg
from RecordingCorpus import RecordingCorpus
from multiprocessing import Pool
from contextlib import closing

warnings.filterwarnings("ignore")
logger = logging.getLogger()
logger.setLevel(logging.INFO)

[NeMo W 2020-10-13 18:34:42 experimental:28] Module <class 'nemo.collections.asr.data.audio_to_text.AudioToCharDataset'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2020-10-13 18:34:42 experimental:28] Module <class 'nemo.collections.asr.data.audio_to_text.AudioToBPEDataset'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2020-10-13 18:34:42 experimental:28] Module <class 'nemo.collections.asr.data.audio_to_text.AudioLabelDataset'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2020-10-13 18:34:42 experimental:28] Module <class 'nemo.collections.asr.data.audio_to_text._TarredAudioToTextDataset'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2020-10-13 18:34:42 experimental:28] Module <class 'nemo.collections.asr.data.audio_to_text.TarredAudioToCharDataset'> is experimental,

In [4]:
C = Cfg('NIST', 8000, 'amharic') 
audio_split_dir=f'{C.build_dir}/audio_split'

In [5]:
!mkdir -p {audio_split_dir}

In [6]:
if 0:
    if __name__ == '__main__':
        with closing(Pool(16)) as pool:
            recordings = RecordingCorpus(C, pool)

    from SplitCorpus import SplitCorpus
    splits=SplitCorpus.transcript_split(C, recordings)

    random.shuffle(splits.artifacts)
    n_samples=len(splits.artifacts)

    n_train = int(0.8*n_samples)
    samples=splits.artifacts
    train_samples=samples[0:n_train]
    test_samples=samples[n_train:]

    for (case, S) in [('train', train_samples), ('test', test_samples)]:
        manifest_fn=f'{C.build_dir}/{case}_manifest.json'
        with open(manifest_fn, 'w', encoding='utf-8') as f_manifest:
            for sample in tqdm(S):
                (_,root,(start,end))=sample.key
                audio = sample.source.value
                duration = sample.source.n_seconds
                transcript = sample.target.value
                audio_path=f'{audio_split_dir}/{root}_{start}_{end}.wav'
                sf.write(audio_path, audio, C.sample_rate)
                metadata = {
                        "audio_filepath": audio_path,
                        "duration": duration,
                        "text": transcript
                    }
                json.dump(metadata, f_manifest)
                f_manifest.write('\n')

In [7]:
model_save_dir='save/nemo_amharic'

In [8]:
!mkdir -p {model_save_dir}

In [9]:
!ls {model_save_dir}

In [10]:
from pytorch_lightning.callbacks import ModelCheckpoint

checkpoint_callback = ModelCheckpoint(
    filepath=model_save_dir+'/amharic_{epoch:02d}-{training_batch_wer:.2f}-{validation_wer:.2f}.pt',
    save_top_k=1,
    verbose=1,
    monitor='training_batch_wer',
    mode='min',
    period=1
)

In [11]:
trainer = pl.Trainer(gpus=[0], max_epochs=1000, amp_level='O1', precision=16, 
                     checkpoint_callback=checkpoint_callback)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]
Using native 16bit precision.


In [12]:
config_path = 'quartznet_15x5_amharic.yaml'
yaml = YAML(typ='safe')
with open(config_path) as f:
    params = yaml.load(f)

In [13]:
params['model']['train_ds']['manifest_filepath'] = f'{C.build_dir}/train_manifest.json'
params['model']['validation_ds']['manifest_filepath'] = f'{C.build_dir}/test_manifest.json'

In [14]:
model = nemo_asr.models.EncDecCTCModel(cfg=DictConfig(params['model']), trainer=trainer)

[NeMo I 2020-10-13 18:35:02 collections:173] Dataset loaded with 8287 files totalling 8.49 hours
[NeMo I 2020-10-13 18:35:02 collections:174] 0 files were filtered totalling 0.00 hours
[NeMo I 2020-10-13 18:35:02 collections:173] Dataset loaded with 2072 files totalling 2.14 hours
[NeMo I 2020-10-13 18:35:02 collections:174] 0 files were filtered totalling 0.00 hours
[NeMo I 2020-10-13 18:35:02 features:241] PADDING: 16
[NeMo I 2020-10-13 18:35:02 features:258] STFT using torch


In [17]:
from glob import glob
models=list(sorted(glob('save/nemo_amharic*.pt')))

In [20]:
if len(models):
    last_fn=models[-1]
    model.restore_from(last_fn)
    print('loaded', last_fn)

[NeMo W 2020-10-13 18:35:37 modelPT:102] Please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: NIST/openasr20_amharic/build/train_manifest.json
    sample_rate: 8000
    labels:
    - ' '
    - ሀ
    - ሁ
    - ሂ
    - ሃ
    - ሄ
    - ህ
    - ሆ
    - ለ
    - ሉ
    - ሊ
    - ላ
    - ሌ
    - ል
    - ሎ
    - ሏ
    - ሐ
    - ሑ
    - ሒ
    - ሔ
    - ሕ
    - መ
    - ሙ
    - ሚ
    - ማ
    - ሜ
    - ም
    - ሞ
    - ሟ
    - ሠ
    - ሡ
    - ሣ
    - ሥ
    - ሦ
    - ረ
    - ሩ
    - ሪ
    - ራ
    - ሬ
    - ር
    - ሮ
    - ሯ
    - ሰ
    - ሱ
    - ሲ
    - ሳ
    - ሴ
    - ስ
    - ሶ
    - ሷ
    - ሸ
    - ሹ
    - ሺ
    - ሻ
    - ሼ
    - ሽ
    - ሾ
    - ሿ
    - ቀ
    - ቁ
    - ቂ
    - ቃ
    - ቄ
    - ቅ
    - ቆ
    - ቋ
    - በ
    - ቡ
    - ቢ
    - ባ
    - ቤ
    - ብ
    - ቦ
    - ቧ
    - ቨ
    - ቪ
    - ቫ
    - ቬ
    - ቭ
    - ተ
    - ቱ
    - ቲ
    - ታ
    - ቴ
    - ት
    - ቶ
    - ቷ
    - ቸ
   

[NeMo I 2020-10-13 18:35:37 features:241] PADDING: 16
[NeMo I 2020-10-13 18:35:37 features:258] STFT using torch
[NeMo I 2020-10-13 18:35:39 modelPT:237] Model EncDecCTCModel was successfully restored from save/nemo_amharic_0087.pt.
loaded save/nemo_amharic_0087.pt


In [ ]:
trainer.fit(model)

[NeMo I 2020-10-13 18:36:11 modelPT:572] Optimizer config = Novograd (
    Parameter Group 0
        amsgrad: False
        betas: [0.8, 0.5]
        eps: 1e-08
        grad_averaging: False
        lr: 0.01
        weight_decay: 0.001
    )
[NeMo I 2020-10-13 18:36:11 lr_scheduler:545] Scheduler "<nemo.core.optim.lr_scheduler.CosineAnnealing object at 0x7f67745054d0>" 
    will be used during training (effective maximum steps = 517938) - 
    Parameters : 
    (warmup_steps: null
    warmup_ratio: null
    min_lr: 0.0
    last_epoch: -1
    max_steps: 517938
    )



  | Name              | Type                              | Params
------------------------------------------------------------------------
0 | preprocessor      | AudioToMelSpectrogramPreprocessor | 0     
1 | encoder           | ConvASREncoder                    | 18 M  
2 | decoder           | ConvASRDecoder                    | 241 K 
3 | loss              | CTCLoss                           | 0     
4 | spec_augmentation | SpectrogramAugmentation           | 0     
5 | _wer              | WER                               | 0     


[NeMo I 2020-10-13 18:36:12 wer:148] 
    
[NeMo I 2020-10-13 18:36:12 wer:149] reference:እ
[NeMo I 2020-10-13 18:36:12 wer:150] decoded  :ቬ


[NeMo I 2020-10-13 18:36:28 wer:148] 
    
[NeMo I 2020-10-13 18:36:28 wer:149] reference:በፊት እምትሰራውን
[NeMo I 2020-10-13 18:36:28 wer:150] decoded  : ጩ
[NeMo I 2020-10-13 18:36:47 wer:148] 
    
[NeMo I 2020-10-13 18:36:47 wer:149] reference:ትምህርት ጀመርክ
[NeMo I 2020-10-13 18:36:47 wer:150] decoded  :
[NeMo I 2020-10-13 18:37:04 wer:148] 
    
[NeMo I 2020-10-13 18:37:04 wer:149] reference:ዊ በጣም አስቸጋሪ ነው ሠራተኛ እኮ በጣም ከባድ ነው አሁን
[NeMo I 2020-10-13 18:37:04 wer:150] decoded  :
[NeMo I 2020-10-13 18:37:22 wer:148] 
    
[NeMo I 2020-10-13 18:37:22 wer:149] reference:ላሳስብሽ ብዬሽ ነው ችላ እንዳትዪ
[NeMo I 2020-10-13 18:37:22 wer:150] decoded  :
[NeMo I 2020-10-13 18:37:39 wer:148] 
    
[NeMo I 2020-10-13 18:37:39 wer:149] reference:ኦኬ እንደት ናት ሲስተር ደህና ነች
[NeMo I 2020-10-13 18:37:39 wer:150] decoded  :
[NeMo I 2020-10-13 18:37:58 wer:148] 
    
[NeMo I 2020-10-13 18:37:58 wer:149] reference:ይሻላል
[NeMo I 2020-10-13 18:37:58 wer:150] decoded  :


In [ ]:
for epoch in range(1,100):
    trainer.fit(model)
    model_fn=f'save/nemo_amharic_{epoch:04d}.pt'
    model.save_to(model_fn)
    !ls -l {model_fn}